In [1]:
import numpy as np
import matplotlib.pyplot as plt

# from keras.applications.densenet import DenseNet201, DenseNet121
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.layers import Dense
from keras import layers, optimizers, models
import keras
from keras import backend as K
# from keras.applications.densenet import preprocess_input
import time

from keras.applications.densenet import DenseNet201
from keras.applications.densenet import preprocess_input
import matplotlib.pyplot as plt

from keras.models import load_model
from keras.applications import imagenet_utils
from keras.applications.imagenet_utils import _preprocess_symbolic_input
from keras.preprocessing import image

import glob
import pickle

FEATURES_LOCATION = '/furniture-data/features/'
F_CORE = 'cnn_features_'

def get_label_from_path(file):
    return file.split('\\')[1].split('.')[0]

def load_data(mode):
    if(mode == 'test'):
        pickle_path = F_CORE + mode
        data = pickle.load(open(FEATURES_LOCATION + pickle_path + '.pkl', 'rb'))
        to_return = {}
        for key, value in list(data.items()):
            to_return[get_label_from_path(key)] = value.reshape(1,-1)
        return to_return, None
    
    pickle_path = F_CORE + mode + '_'
    
    data = {}
    for i in range(1,129):
        data[i] = pickle.load(open(FEATURES_LOCATION + pickle_path + str(i) + '.pkl', 'rb'))
        
    X = []
    y = []
    for key, value in list(data.items()):
        the_class = key
        features = np.array(list(value.values()))
        for feature in features:
            y.append(the_class)
            X.append(feature)

    return np.array(X), np.array(y)

/home/1nnuser/lsdaenv/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
X, y = load_data('train')
X_val, y_val = load_data('valid')

from sklearn.utils import class_weight
class_weight = class_weight.compute_class_weight('balanced', np.unique(y), y)

In [20]:
target_size = (224,224)
# validation_dir = './data/valid'
# train_dir = './data/train'
validation_dir = '/furniture-data/valid'
train_dir = '/furniture-data/train'

train_datagen = ImageDataGenerator(
        rotation_range=10,
        width_shift_range=0.2,
        height_shift_range=0.2,
        horizontal_flip=True,
        preprocessing_function=preprocess_input
      )
 
validation_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
 
# Change the batchsize according to your system RAM
train_batchsize = 100
val_batchsize = 10
 
train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=target_size,
        batch_size=train_batchsize,
        class_mode='categorical')
 
validation_generator = validation_datagen.flow_from_directory(
        validation_dir,
        target_size=target_size,
        batch_size=val_batchsize,
        class_mode='categorical',
        shuffle=False)

Found 192171 images belonging to 128 classes.
Found 6309 images belonging to 128 classes.


In [15]:
cnn = DenseNet201(weights='imagenet', include_top=False, pooling='max')

for layer in cnn.layers:
    layer.trainable = False

In [17]:
import keras

model = keras.models.Sequential()
model.add(keras.layers.InputLayer(input_shape=(224,224,3)))
model.add(cnn)
# model.add(keras.layers.MaxPool2D(pool_size=(3,3)))
# model.add(keras.layers.Flatten(name='flat'))
model.add(layers.Dense(1024, activation='relu', name='hidden_1024', input_shape=(2048,)))
model.add(layers.Dropout(0.5, name='hidden_1024_dropout'))
model.add(layers.Dense(128, activation='softmax', name='classification'))

In [18]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_8 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
densenet201 (Model)          (None, 1920)              18321984  
_________________________________________________________________
hidden_1024 (Dense)          (None, 1024)              1967104   
_________________________________________________________________
hidden_1024_dropout (Dropout (None, 1024)              0         
_________________________________________________________________
classification (Dense)       (None, 128)               131200    
Total params: 20,420,288
Trainable params: 2,098,304
Non-trainable params: 18,321,984
_________________________________________________________________


In [21]:
# Fine-tuning optimizer
# optm = optimizers.RMSprop(lr=1e-4)
optm = optimizers.Adam()

model.compile(loss='categorical_crossentropy',
              optimizer=optm,
              metrics=['acc'])

# Train the model
history = model.fit_generator(
      train_generator,
      steps_per_epoch=train_generator.samples/train_generator.batch_size/10,
      epochs=5*10,
      class_weight=class_weight,
      validation_data=validation_generator,
      validation_steps=validation_generator.samples/validation_generator.batch_size/2,
      verbose=1,
      callbacks=[keras.callbacks.ModelCheckpoint('densenet.h5', monitor='val_acc', verbose=2, save_best_only=True, mode='auto', period=1)])
 
# Save the model
model.save('densenet_final.h5')

Epoch 1/50
193/192 [==============================] - 1264s 7s/step - loss: 4.1869 - acc: 0.0683 - val_loss: 3.6373 - val_acc: 0.1595

Epoch 00001: val_acc improved from -inf to 0.15949, saving model to densenet.h5
Epoch 2/50
193/192 [==============================] - 1239s 6s/step - loss: 3.6634 - acc: 0.1377 - val_loss: 3.1983 - val_acc: 0.2140

Epoch 00002: val_acc improved from 0.15949 to 0.21399, saving model to densenet.h5
Epoch 3/50
193/192 [==============================] - 1180s 6s/step - loss: 3.4350 - acc: 0.1632 - val_loss: 2.7533 - val_acc: 0.2978

Epoch 00003: val_acc improved from 0.21399 to 0.29778, saving model to densenet.h5
Epoch 4/50
193/192 [==============================] - 1198s 6s/step - loss: 3.3070 - acc: 0.1773 - val_loss: 2.7175 - val_acc: 0.2795

Epoch 00004: val_acc did not improve
Epoch 5/50
193/192 [==============================] - 1244s 6s/step - loss: 3.2255 - acc: 0.1890 - val_loss: 2.6868 - val_acc: 0.3092

Epoch 00005: val_acc improved from 0.29778

KeyboardInterrupt: 

In [ ]:
# optm = optimizers.RMSprop(lr=1e-4)
optm = optimizers.Adam(lr=0.00001)

model.compile(loss='categorical_crossentropy',
              optimizer=optm,
              metrics=['acc'])

# Train the model
history = model.fit_generator(
      train_generator,
      steps_per_epoch=train_generator.samples/train_generator.batch_size/10,
      epochs=5*10,
      class_weight=class_weight,
      validation_data=validation_generator,
      validation_steps=validation_generator.samples/validation_generator.batch_size/2,
      verbose=1,
      callbacks=[keras.callbacks.ModelCheckpoint('densenet.h5', monitor='val_acc', verbose=2, save_best_only=True, mode='auto', period=1)])
 
# Save the model
model.save('densenet_final.h5')

Epoch 1/50
193/192 [==============================] - 619s 3s/step - loss: 2.9318 - acc: 0.2383 - val_loss: 2.4789 - val_acc: 0.3108

Epoch 00001: val_acc improved from -inf to 0.31076, saving model to densenet.h5
Epoch 2/50
161/192 [========================>.....] - ETA: 1:23 - loss: 2.9437 - acc: 0.2376